In [1]:
import torch
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scienceplots
import imageio.v2 as imageio
import glob

from matplotlib import animation
from matplotlib.animation import PillowWriter

plt.style.use(['science', 'notebook'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## TWO STREAM INSTABILITY USING TORCH

In [2]:
L  = 100    # Domain of the solution 0 <= x <= L  (in Debye lengths)
N  = 25000  # Number of electrons
J  = 1000   # Number of grid-points
vb = 5      # Beam velocity
n0 = N/L    # ion number density
dx = L/J

dt = 0.1    # time step  (in inverse plasma frequencies)
t_max = 150  # such that 0 <= t <= t_max
timesteps = int(t_max / dt)


# Check input parameters make sence:
if (N < 1) | (J < 2) | (L <= 0.) | (vb <= 0.) | (dt <= 0.) | (t_max <= 0.) | ((int (t_max / dt) / 10) < 1):
    print("Error - invalid input parameters")
   

| Goal                     | NumPy                                | PyTorch                                |
|--------------------------|---------------------------------------|----------------------------------------|
| Uniform sample [a, b)    | `np.random.uniform(a, b, size)`       | `(b - a) * torch.rand(size) + a`       |
| Normal sample (mean, σ)  | `np.random.normal(mean, std, size)`   | `torch.normal(mean, std, size)`        |
| Random ints [a, b)       | `np.random.randint(a, b, size)`       | `torch.randint(a, b, size)`            |


The following code takes a lot of time to run, despite it is the torch version of the numpy implementation. 

```Python

# Initial positions
r0 = torch.rand(N, device=device) * L

# Initial velocities with rejection sampling
def sample_velocity(v_b, tails_factor):
    f_max = 0.5 * (1.0 + np.exp(-2 * v_b**2))  # ok to use np.exp since v_b is a scalar
    vmin, vmax = -tails_factor * v_b, tails_factor * v_b

    velocities = torch.empty(N, device=device)
    i = 0
    while i < N:
        v_ = (vmax - vmin) * torch.rand(1, device=device) + vmin  # torch version
        beam_shape = 0.5 * (torch.exp(-0.5 * (v_ - v_b)**2) + torch.exp(-0.5 * (v_ + v_b)**2))
        gamma = torch.rand(1, device=device) * f_max
        if gamma <= beam_shape:
            velocities[i] = v_
            i += 1
    return velocities

v0 = sample_velocity(vb, 4)
```

This is because NumPy is highly optimized for scalar and small-batch CPU operations, and its core is implemented in C — so for modest sizes like N = 25,000, rejection sampling in NumPy can outperform PyTorch if you're not using vectorized operations. Thus, it's simpler to implementin numpy and then convert. 

### USING BATCHES

However, the original rejection sampling does one sample at a time — **very inefficient**, especially on a GPU where batch operations are massively faster.

By generating many random samples in batches, we:

1. Reduce the number of iterations in the while loop.

2. Allow operations like `torch.exp`, `torch.rand`, and masking to be applied in parallel over large arrays.

3. **Use the GPU (or SIMD instructions on CPU) as they were designed to be used** — on big chunks of data.

So batch_size controls how many samples we generate at once, hoping that many of them will be accepted in a single round.

**WHICH BATCH SIZE?** This is a heuristic value, for a real-world application or a more serious scenario, it is recommended to work on a benchmark script to test different batch sizes and chose the optimal value. For now, it will be set `batch_size = 100000` as a heuristic value: big enough to efficiently use the GPU, small enough to avoid memory overflow.


| Scenario                   | NumPy (loop)    | Torch (vectorized)            |
| -------------------------- | --------------- | ----------------------------- |
| Small N, CPU-only          | Probably faster | Slightly slower               |
| Large N (e.g., 100k+), CPU | About equal     | Faster with batching          |
| Large N, **GPU** available | ❌ CPU-bound     | ✅ Hugely faster with batching |




In [ ]:
# Initial positions
r0 = torch.rand(N, device=device) * L

def sample_velocity(v_b, tails_factor, batch_size=100000):
    f_max = 0.5 * (1.0 + np.exp(-2 * v_b**2))
    vmin, vmax = -tails_factor * v_b, tails_factor * v_b

    velocities = torch.empty(N, device=device)
    filled = 0

    while filled < N:
        v_ = (vmax - vmin) * torch.rand(batch_size, device=device) + vmin
        beam_shape = 0.5 * (torch.exp(-0.5 * (v_ - v_b)**2) + torch.exp(-0.5 * (v_ + v_b)**2))
        gamma = torch.rand(batch_size, device=device) * f_max
        accepted = v_[gamma <= beam_shape]

        num_to_fill = min(accepted.shape[0], N - filled)
        velocities[filled:filled+num_to_fill] = accepted[:num_to_fill]
        filled += num_to_fill

    return velocities
v0 = sample_velocity(vb,4)
velocity_tags = abs(v0 - vb) < abs(v0 + vb) # False = left going

`scatter_add_` handles repeated indices like `np.add.at`; It's fast, **parallel**, a

Next, we will use `torch.zeros` instead of `torch.empty`, why? Because we are doing an addition of the previous values. 


| Function      | Use case                                                                    |
| ------------- | --------------------------------------------------------------------------- |
| `torch.empty` | You're **immediately overwriting** all values (e.g., `velocities[:] = ...`) |
| `torch.zeros` | You're **accumulating**, or using values **before** assigning all of them   |


In [8]:
def compute_density(r_):
    ne_ = torch.zeros(J, device=device)
    j_  = torch.floor(r_ / dx).long()
    f   = (r_ / dx) - j_ # fractional offset
    
    # Right: weight = f / dx
    ne_.scatter_add_(0, (j_+1)%J, f / dx)
    
    # Left: weight = (1 - f) / dx
    ne_.scatter_add_(0, j_%J, (1 - f) / dx)
    
    return ne_

Next: Poisson solver. 

The existing NumPy version uses np.fft.fft and np.fft.ifft, which are efficient and totally valid.

There are two options:

| Option | Approach            | Use it if...                                         |
| ------ | ------------------- | ---------------------------------------------------- |
| ✅ 1    | **Keep NumPy FFTs** | You're OK with a NumPy dependency, or not on GPU yet |
| ⚡ 2    | `torch.fft.fft`     | You want full GPU usage (no CPU↔GPU transfers)       |


Option 2: 
* Uses `torch.fft.fft`, `ifft`, and `fftfreq` equivalents.
* All operations stay on `device`, so no CPU-GPU ping-pong.
* Just like in `NumPy`, we set `phi_hat[0] = 0` to avoid the DC offset.

In [ ]:
def poisson_solver(ne_):
    rho = ne_ / n0 - 1 # normalized charge density
    
    # FFT of rho (torch.fft returns complex tensor)
    rho_hat = torch.fft.fft(rho)
    
    # Wavenumbers (same as np.fft.fftfreq)
    k = 2 * np.pi * torch.fft.fftfreq(J, d=L/J).to(device)
    
    # Avoid division by zero
    phi_hat = torch.zeros_like(rho_hat)
    phi_hat[1:] = -rho_hat[1:] / (k[1:]**2)
    phi_hat[0] = 0.0 + 0.0j
    
    # Inverse FFT to get potential phi(x)
    phi = torch.fft.ifft(phi_hat).real  # result should be real-valued
    
    return phi

Electric field $\rightarrow$ getting rid of the `for` loop.

For this, we will be using `torch.roll`. Why?
* `torch.roll` is the perfect tool for periodic boundaries (% J becomes unnecessary)
* Vectorized, GPU-ready, and avoids loops.

`torch.roll(input, shifts, dims=None)` → Tensor

Roll the tensor `input` along the given dimension(s). Elements that are shifted beyond the last position are re-introduced at the first position. If dims is None, the tensor will be flattened before rolling and then restored to the original shape.

Example 

```Python
>>> x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]).view(4, 2)
>>> x
tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])
>>> torch.roll(x, 1)
tensor([[8, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
>>> torch.roll(x, 1, 0)
tensor([[7, 8],
        [1, 2],
        [3, 4],
        [5, 6]])
>>> torch.roll(x, -1, 0)
tensor([[3, 4],
        [5, 6],
        [7, 8],
        [1, 2]])
>>> torch.roll(x, shifts=(2, 1), dims=(0, 1))
tensor([[6, 5],
        [8, 7],
        [2, 1],
        [4, 3]])
```

In [ ]:
def electric_field(phi_):
    # Use periodic shifting
    phi_left  = torch.roll(phi_, shifts=1, dims=0)
    phi_right = torch.roll(phi_, shifts=-1, dims=0)
    
    E_grid = (phi_left - phi_right) / (2 * dx)
    return E_grid

Let's have a look at the electric field perceived by the particles. 

In [ ]:
def particle_electric_field(r_, E_):
    j_ = torch.floor(r_ / dx).long()
    f_ = (r_ / dx) - j_

    E_particles = (1 - f_)*E_[ j_ % J ] + f_*E_[ (j_ + 1) % J ]
    return E_particles

Let us calculate the derivatives from the equations of motion to be fed as functions for the Runge-Kutta solutions. 

In [12]:
def derivative_r_v(y_):
    """
    Returns dy/dt, with f = [r0,r1, ..., rN,v0,v1,...,vN]
    Such that dy/dt = [dr/dt, dv/dt] = [v(r_i), -E(r_i)]
    For notation dy/dt --> dy [tuple]
    """
    
    r_ = y_[:N]
    v_ = y_[N:] 
    
    # 1. Compute the density
    ne = compute_density(r_)
    
    # 2. Solve Poisson's equation
    phi = poisson_solver(ne)
    
    # 3. Compute the Electric field in the grid
    E_grid = electric_field(phi)
    
    # 4. Compute the Electric field per particle
    E_particles = particle_electric_field(r_,E_grid)
    
    return torch.cat([v_, -E_particles])


def RungeKutta4(y_):
    """
    dy = dy/dt = [dr/dt, dv/dt] = [v(r_i), -E(r_i)]
    """
    
    # f(x,y)    
    k1 = derivative_r_v( y_)
    k2 = derivative_r_v( y_ + 0.5*dt*k1)
    k3 = derivative_r_v( y_ + 0.5*dt*k2)
    k4 = derivative_r_v( y_ + dt*k3)
    
    return y_ + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)

In [ ]:
y = torch.cat([r0, v0])
for step in range(timesteps):
    y = RungeKutta4(y)
    # Enforce periodic boundary conditions
    y[:N] = y[:N] % L